In [2]:
import numpy as np
import pandas as pd

import torch

In [3]:
from gluonts.dataset.multivariate_grouper import MultivariateGrouper
from gluonts.dataset.repository.datasets import dataset_recipes, get_dataset
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import MultivariateEvaluator

from pts.model.deepar import DeepAREstimator
from pts.dataset.repository.datasets import dataset_recipes
from pts.modules import StudentTOutput

2023-03-09 10:54:21.219616: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2023-03-09 10:54:21.978515: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2023-03-09 10:54:21.978586: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory
2023-03-09 10:54:21.978593: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed proper

## Prepeare data set

In [4]:
dataset = get_dataset("solar_nips", regenerate=False)

In [5]:
dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat_0', cardinality='137')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [6]:
train_grouper = MultivariateGrouper(max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

test_grouper = MultivariateGrouper(num_test_dates=int(len(dataset.test)/len(dataset.train)), 
                                   max_target_dim=int(dataset.metadata.feat_static_cat[0].cardinality))

In [7]:
dataset_train = train_grouper(dataset.train)
dataset_test = test_grouper(dataset.test)

## Evaluator

In [8]:
evaluator = MultivariateEvaluator(quantiles=(np.arange(20)/20.0)[1:],
                                  target_agg_funcs={'sum': np.sum})

## `DeepVAR`

In [17]:
estimator = DeepAREstimator(
    distr_output=StudentTOutput(dim=int(dataset.metadata.feat_static_cat[0].cardinality)),
    input_size=int(dataset.metadata.feat_static_cat[0].cardinality),
    hidden_size=64,
    num_layers=3,
    dropout_rate=0.1,
    lags_seq=[1, 24*7],
  
    prediction_length=dataset.metadata.prediction_length,
    context_length=dataset.metadata.prediction_length,
    freq=dataset.metadata.freq,
    scaling="std",
    trainer_kwargs=dict(max_epochs=50, accelerator='gpu', devices='1'),
)

In [18]:
predictor = estimator.train(dataset_train, cache_data=True)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/home/kashif/.env/pytorch/lib/python3.10/site-packages/pytorch_lightning/trainer/configuration_validator.py:108: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
You are using a CUDA device ('NVIDIA GeForce RTX 3090 Ti') that has Tensor Cores. To properly utilize them, you should set `torch.set_float32_matmul_precision('medium' | 'high')` which will trade-off precision for performance. For more details, read https://pytorch.org/docs/stable/generated/torch.set_float32_matmul_precision.html#torch.set_float32_matmul_precision
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name  | Type        | Params | In sizes | Out sizes        
---------------------------------------------------------------------
0 | model | DeepARModel | 252 K  | ?        | [1, 100, 24, 137]
-------

Training: 0it [00:00, ?it/s]

Epoch 0, global step 50: 'train_loss' reached 683.26031 (best 683.26031), saving model to '/home/kashif/pytorch-ts/examples/lightning_logs/version_37/checkpoints/epoch=0-step=50.ckpt' as top 1
Epoch 1, global step 100: 'train_loss' reached 567.28973 (best 567.28973), saving model to '/home/kashif/pytorch-ts/examples/lightning_logs/version_37/checkpoints/epoch=1-step=100.ckpt' as top 1
Epoch 2, global step 150: 'train_loss' reached 532.46063 (best 532.46063), saving model to '/home/kashif/pytorch-ts/examples/lightning_logs/version_37/checkpoints/epoch=2-step=150.ckpt' as top 1
Epoch 3, global step 200: 'train_loss' reached 517.28900 (best 517.28900), saving model to '/home/kashif/pytorch-ts/examples/lightning_logs/version_37/checkpoints/epoch=3-step=200.ckpt' as top 1
Epoch 4, global step 250: 'train_loss' reached 507.24280 (best 507.24280), saving model to '/home/kashif/pytorch-ts/examples/lightning_logs/version_37/checkpoints/epoch=4-step=250.ckpt' as top 1
Epoch 5, global step 300: '

`Trainer.fit` stopped: `max_epochs=50` reached.


In [19]:
forecast_it, ts_it = make_evaluation_predictions(dataset=dataset_test,
                                             predictor=predictor,
                                             num_samples=100)
forecasts = list(forecast_it)
targets = list(ts_it)
agg_metric, _ = evaluator(targets, forecasts, num_series=len(dataset_test))

Running evaluation: 7it [00:00, 88.03it/s]
Running evaluation: 7it [00:00, 98.35it/s]
Running evaluation: 7it [00:00, 99.19it/s]
Running evaluation: 7it [00:00, 97.87it/s]
Running evaluation: 7it [00:00, 98.91it/s]
Running evaluation: 7it [00:00, 99.60it/s]
Running evaluation: 7it [00:00, 92.95it/s]
Running evaluation: 7it [00:00, 99.10it/s]
Running evaluation: 7it [00:00, 99.15it/s]
Running evaluation: 7it [00:00, 98.41it/s]
Running evaluation: 7it [00:00, 97.88it/s]
Running evaluation: 7it [00:00, 98.10it/s]
Running evaluation: 7it [00:00, 94.85it/s]
Running evaluation: 7it [00:00, 101.79it/s]
Running evaluation: 7it [00:00, 101.44it/s]
Running evaluation: 7it [00:00, 101.46it/s]
Running evaluation: 7it [00:00, 100.72it/s]
Running evaluation: 7it [00:00, 100.74it/s]
Running evaluation: 7it [00:00, 101.57it/s]
Running evaluation: 7it [00:00, 101.28it/s]
Running evaluation: 7it [00:00, 101.20it/s]
Running evaluation: 7it [00:00, 100.77it/s]
Running evaluation: 7it [00:00, 101.12it/s]
R

### Metrics

In [20]:
print("CRPS: {}".format(agg_metric['mean_wQuantileLoss']))
print("ND: {}".format(agg_metric['ND']))
print("NRMSE: {}".format(agg_metric['NRMSE']))
print("MSE: {}".format(agg_metric['MSE']))

CRPS: 0.48857047034606677
ND: 0.5834631932625387
NRMSE: 1.1737168856721258
MSE: 1306.7860587706446


In [21]:
print("CRPS-Sum: {}".format(agg_metric['m_sum_mean_wQuantileLoss']))
print("ND-Sum: {}".format(agg_metric['m_sum_ND']))
print("NRMSE-Sum: {}".format(agg_metric['m_sum_NRMSE']))
print("MSE-Sum: {}".format(agg_metric['m_sum_MSE']))

CRPS-Sum: 0.4911344613270915
ND-Sum: 0.5287813955218926
NRMSE-Sum: 0.9606235665322241
MSE-Sum: 16429540.476190476
